In [30]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama

import os
from dotenv import load_dotenv

import os
from exa_py import Exa

# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

True

In [54]:
llm = Ollama(model="llama3.2:1b") 

## Sumariza a pergunta 

In [24]:
def build_summarizer_agent():
    llm = Ollama(model="llama3.2:1b")  # ou outro modelo, se necessário

    prompt = PromptTemplate(
        input_variables=["question"],
        template="""
        Dada a pergunta abaixo, identifique o **principal nome da pessoa, obra ou evento** que é o foco central da questão. Forneça **apenas esse nome**, sem qualquer outra parte da pergunta, detalhes adicionais ou outros nomes relacionados.

        Pergunta: {question}

        Resposta direta:
        """
    )

    chain = LLMChain(llm=llm, prompt=prompt)
    return chain



In [32]:
# Criar o summarizer
summarizer = build_summarizer_agent()

# Testar com uma pergunta
question = "Me fale do chico césar?"
result_key = summarizer.run(question)

# Mostrar o resultado
print(f"Consulta resumida: {result_key}")

Consulta resumida: Chico César


## Pesquisa e scrapa a url da enciclopédia

In [47]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus

def get_itau_link_and_preview(query: str) -> dict:
    base_url = "https://enciclopedia.itaucultural.org.br"
    search_url = f"{base_url}/busca?q={quote_plus(query)}"

    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        }
        response = requests.get(search_url, headers=headers)

        if response.status_code != 200:
            return {"link": None, "preview": f"Erro ao acessar a Enciclopédia (status {response.status_code})."}

        soup = BeautifulSoup(response.text, "html.parser")
        article = soup.find("article", class_="row no-gutters")

        if not article:
            return {"link": None, "preview": "Nenhuma entrada encontrada."}

        a_tag = article.find("a", href=True)
        subtitle_div = article.find("div", class_="subtitle")

        if a_tag:
            href = a_tag["href"]
            full_link = f"{base_url}{href}"
            preview_text = subtitle_div.text.strip() if subtitle_div else "Nenhum subtítulo disponível."
            return {"link": full_link, "preview": preview_text}

        return {"link": None, "preview": "Não foi possível encontrar o link principal."}

    except Exception as e:
        return {"link": None, "preview": f"Erro na requisição: {str(e)}"}

In [48]:
result = get_itau_link_and_preview(result_key)
print(result)

{'link': 'https://enciclopedia.itaucultural.org.br/pessoas/10620-chico-cesar', 'preview': 'Registro fotográfico Marcus Leoni'}


In [50]:
scrap_link = result["link"]
scrap_link 

'https://enciclopedia.itaucultural.org.br/pessoas/10620-chico-cesar'

## Retorna o conteúdo da enciclopédia

In [51]:

# Verificar se a chave de API foi corretamente configurada
exa_key = os.getenv('EXA_API_KEY')

# Função para obter os conteúdos das URLs
def get_exa_contents(urls: list) -> str:
    """
    Função para buscar conteúdos de uma lista de URLs utilizando a Exa API.
    Retorna o conteúdo das URLs.

    Parameters:
    - urls (list): Lista de URLs para buscar conteúdo.

    Returns:
    - str: Conteúdo das URLs solicitadas.
    """
    # Obtenha a chave da variável de ambiente
    exa_key = os.getenv('EXA_API_KEY')

    # Verifique se a chave de API foi carregada corretamente
    if exa_key is None:
        return "Erro: A chave de API Exa não foi definida."

    try:
        # Inicialize o cliente Exa com a chave de API
        exa = Exa(api_key=exa_key)

        # Obter conteúdos das URLs passadas
        results = exa.get_contents(urls)

        # Retornar os resultados em formato de string
        return str(results)

    except Exception as e:
        # Em caso de erro, retornar mensagem de erro
        return f"Erro ao obter conteúdos: {str(e)}"

# Supondo que scrap_link já tenha o valor do link (como 'https://...')
# Correção na chamada da função:
result = get_exa_contents([scrap_link])

# Exibir o conteúdo retornado
print(result)


Title: Chico César
URL: https://enciclopedia.itaucultural.org.br/pessoas/10620-chico-cesar
ID: https://enciclopedia.itaucultural.org.br/pessoas/10620-chico-cesar
Score: None
Published Date: 2024-10-25T00:00:00.000Z
Author: Instituto Itaú Cultural
Image: https://editor-enciclopedia-public-prd.s3.sa-east-1.amazonaws.com/z88jxhyteqwt3ggbtzoccnymspi9
Favicon: https://enciclopedia.itaucultural.org.br/packs/media/images/favicon-af3d8fe3b70e783e3e7ab0189d538b4f.ico
Extras: None
Subpages: None
Text: Texto 
 
 
 
 
 Francisco César Gonçalves (Catolé do Rocha, Paraíba, 1964). Cantor, compositor, violonista e poeta. Influenciado pela tropicália e pela vanguarda paulistana, mistura ritmos ao transitar entre elementos da cultura popular e canções românticas. 
 Na adolescência, participa dos grupos musicais The Snakes, Trio Mirim, Super Som Mirim e Ferradura. Aos 13 anos, apresenta sua primeira composição, o samba “Quando Chega o Carnaval”. Em 1980, transfere-se para a capital João Pessoa, onde se f

## Responde a pergunta a partir do conteúdo 

In [55]:
def format_large_content(content: str, question: str, max_length: int = 1500):
    """
    Gera uma resposta concisa para a pergunta do usuário com base no conteúdo fornecido.

    Parameters:
    - content (str): O conteúdo extraído da página da Enciclopédia Itaú Cultural.
    - question (str): A pergunta original do usuário.
    - max_length (int): Comprimento máximo do conteúdo a ser considerado (para evitar textos muito longos).

    Returns:
    - str: Uma resposta gerada com base no conteúdo e na pergunta.
    """
    # Se o conteúdo for muito grande, pega apenas os primeiros tokens
    if len(content) > max_length:
        content = content[:max_length] + "..."

    prompt = PromptTemplate(
        input_variables=["content", "question"],
        template="""
        Com base no seguinte conteúdo da Enciclopédia Itaú Cultural:
        {content}

        Responda à seguinte pergunta do usuário da forma mais concisa e relevante possível:
        {question}

        Resposta:
        """
    )

    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run({"content": content, "question": question})
    return response.strip()

In [56]:
respota_final = format_large_content(result, question)

In [57]:
respota_final

'O Chico César é um músico, compositor e poeta português de católico, nascido em Catolé do Rocha, Paraíba, em 1964.'